In [14]:
import pandas as pd
import numpy as np
from pathlib import Path
import os
from Data import*

bom_df = pd.read_excel("Bill of Material (mrp.bom)updated.xlsx")

DataMgr = DataManager()
DataMgr.getResources().clear()
DataMgr.getProducts().clear()

def MakeProduct (dbmgr,product):

    if (product.find("[")>-1) and (product.find("]")>-1):
        productnumber = product[product.find("[")+1:product.find("]")];
        productname = product[product.find("]")+1:];
        if not "["+productnumber+"]_"+productname in dbmgr.getProducts():
            newproduct = Product(len(dbmgr.getProducts()),"["+productnumber+"]_"+productname,productnumber,0);
            return newproduct
        else:
            return dbmgr.getProducts()["["+productnumber+"]_"+productname]
    else:
        if not product in dbmgr.getProducts():
            newproduct = Product(len(dbmgr.getProducts()),product,product,0);
            return newproduct
        else:
            return dbmgr.getProducts()[product]

def MakeOperation(dbmgr,operation,prod,resource):
    myresource=None;
    if not resource in dbmgr.getResources():
        if "manual" in resource.split():
            myresource = Resource(len(dbmgr.getResources()),"Manual",resource,16);
            dbmgr.getResources()[resource]=myresource;
        else:
            myresource = Resource(len(dbmgr.getResources()),"Machine",resource,16);
            dbmgr.getResources()[resource]=myresource;
    else:
        myresource = dbmgr.getResources()[resource]; 

    optimestr= r['BoM Lines/Operations/Duration'];

    optime = 0
    
    if isinstance(optimestr, (int, float)):
        optime = float(1/60)*float(optimestr) # time assumed to be minute..

    newopr = Operation(len(dbmgr.getOperations()),operation+"_"+prod.getName(),optime)

    if myresource !=None:
        myresource.getOperations().append(newopr)
        newopr.getRequiredResources().append(myresource)

    if not newopr.getName() in dbmgr.getOperations():
        dbmgr.getOperations()[newopr.getName()]=newopr;
        
    return newopr
    
nopass = 0;
for i,r in bom_df.iterrows():

    productrow = False;
    if nopass > 0:
        nopass -=1
        continue

    if isinstance(r['Product'],str):
        if not pd.isnull(r['Product']):
            productrow = True;

    if productrow == True:
        newproduct = MakeProduct(DataMgr,r['Product'])
        DataMgr.getProducts()[newproduct.getName()] = newproduct;

        if not pd.isnull(r['BoM Lines/Component']):
            
            rawprod = MakeProduct(DataMgr,r['BoM Lines/Component'])

            quantity = r['BoM Lines/Quantity']

            newproduct.getMPredecessors()[rawprod] = quantity
            newproduct.getPredecessors().append(rawprod)
            rawprod.setSuccessor(newproduct)

            DataMgr.getProducts()[rawprod.getName()] = rawprod

        j = i
        while j < len(bom_df):
            if not pd.isnull(bom_df.iloc[j,bom_df.columns.get_loc('Product')]) and j > i:
                break
            #print("Operation: ",bom_df.iloc[j,bom_df.columns.get_loc('BoM Lines/Operations')],"-",bom_df.iloc[j,bom_df.columns.get_loc('Reference')])

            if not pd.isnull(bom_df.iloc[j,bom_df.columns.get_loc('BoM Lines/Operations')]):
                myopr = MakeOperation(DataMgr,bom_df.iloc[j,bom_df.columns.get_loc('BoM Lines/Operations')],newproduct,bom_df.iloc[j,bom_df.columns.get_loc('BoM Lines/Operations/Work Center')])
                newproduct.getOperations().append(myopr)

               
   
            j+=1 
        nopass=len(newproduct.getOperations())-1
       
        i = j-1
        #print("i",i,"j",j)
    else:
        print(">>>>>>>>>>>> No Product definition: ", isinstance(r['Product'],str),pd.isnull(r['Product']))
        print("Product value: ",r['Product'])
        print(r)
        

print('Nr Products: ',len(DataMgr.getProducts()))
print('Nr Resources: ',len(DataMgr.getResources()))
print('Nr Operations: ',len(DataMgr.getOperations()))

    
    

    
    
        

    

Nr Products:  561
Nr Resources:  15
Nr Operations:  984
